In [1]:
import pandas as pd
import re
import spacy
import plac
from pathlib import Path
from spacy.util import minibatch, compounding
from __future__ import unicode_literals, print_function
import csv

import funconstructor as fc
import train as tr

import matplotlib.pyplot as plt
import seaborn as sns
from pylab import savefig
import pickle
import random

In [2]:
df = pd.read_csv('input/#jokermovie.csv',)

FileNotFoundError: [Errno 2] File b'input/#jokermovie.csv' does not exist: b'input/#jokermovie.csv'

In [ ]:
categories = {
    'CHARACTERS': ['character', 'characters', 'Arthur Fleck', 'Murray Franklin', 'Sophie Dumond', 'Penny Fleck', 'mother', 'mum'],
    'TALENT': ['actor', 'actress', 'actors', 'talent', 'director', 'Joaquin Phoenix', '#JoaquinPhoenix', 'Robert De Niro', 'Zazie Beetz', 'Frances Conroy', 'Todd Philips'],
    'PLOT': ['plot', 'frame', 'scene', 'scenes'],
    'TICKETS': ['ticket', 'tickets', 'presale'],
    'SOUNDTRACK': ['music', 'song'],
    'SPECIAL EFFECTS': ['FX', 'visual effects', 'graphics', 'postproduction'],
    'COSTUME DESIGN': ['costum', 'costumes', 'costume designer'],
    'HYPE': ['hype', 'hyyyype', 'amazing', 'awesome', 'desire', 'desires', 'premiere', 'Cant wait'],
    'BOX OFFICE': ['box office'],
    'REVIEWS': ['review', 'reviews'],
    'TRAILER':['trailer', 'trailers']
}

In [ ]:
main(training_data,output_dir='output/model')

In [ ]:
@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int),
)
def main(TRAIN_DATA, model=None, output_dir=None, n_iter=100):
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # reset and initialize the weights randomly – but only if we're
        # training a new model
        if model is None:
            nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4.0, 32.0, 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    losses=losses,
                )
            print("Losses", losses)

    # test the trained model
    for text, _ in TRAIN_DATA:
        doc = nlp(text)
        print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
        print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
    if output_dir is not None:
        output_dir = Path(output_dir)
        if not output_dir.exists():
            output_dir.mkdir()
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

        # test the saved model
        print("Loading from", output_dir)
        nlp2 = spacy.load(output_dir)
        for text, _ in TRAIN_DATA:
            doc = nlp2(text)
            print("Entities", [(ent.text, ent.label_) for ent in doc.ents])
            print("Tokens", [(t.text, t.ent_type_, t.ent_iob) for t in doc])

In [ ]:
training_data = train.trainData(df, categories)

In [ ]:
def categoryColumn(path, df):
    #Predict category while iterate through text serie and save the final DataFrame 
    nlp = spacy.load(path)

    for index, row in df.iterrows():
        doc = nlp(row['text'])
        for ent in doc.ents:
            df.loc[index, 'category'] = [(ent.label_)]
    
    return df.to_csv('output/predicted.csv')

In [ ]:
categoryColumn('output/model', df) #genera predicted.csv

In [ ]:
dfp = pd.read_csv('output/predicted.csv')
dfp.head()

In [4]:
fc.dataReady('../input/#jokermovie.csv', 'data.csv')

Created blank 'en' model
Losses {'ner': 1232.661986104}
Losses {'ner': 62.461925289545704}
Losses {'ner': 57.412915389453204}
Losses {'ner': 53.662067616383354}
Losses {'ner': 54.610824534353235}
Losses {'ner': 56.78517680447648}
Losses {'ner': 60.21691312458194}
Losses {'ner': 63.37937429222467}
Losses {'ner': 61.56015200280731}
Losses {'ner': 57.51507721997719}
Losses {'ner': 46.96218017186229}
Losses {'ner': 43.683734010261595}
Losses {'ner': 40.38704019832334}
Losses {'ner': 42.45843199596156}
Losses {'ner': 33.01917343664132}
Losses {'ner': 36.9655876446863}
Losses {'ner': 35.4923183757515}
Losses {'ner': 33.62574786055501}
Losses {'ner': 24.102654317582548}
Losses {'ner': 37.000233158351435}
Losses {'ner': 24.642552396920372}
Losses {'ner': 21.19048298150555}
Losses {'ner': 28.78505494758695}
Losses {'ner': 17.693760896240345}
Losses {'ner': 20.771701808037868}
Losses {'ner': 18.353280618462232}
Losses {'ner': 11.133308669924523}
Losses {'ner': 19.864132166397688}
Losses {'ner': 

Entities []
Tokens [('Open', '', 2), ('bookings', '', 2), ('in', '', 2), ('#', '', 2), ('Hyderabad', '', 2), ('for', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('@warnerbrosindia', '', 2)]
Entities []
Tokens [('I', '', 2), ('have', '', 2), ('a', '', 2), ('new', '', 2), ('favorite', '', 2), ('joker', '', 2), ('@jokermovie', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('pic.twitter.com/S4wOE3lmQF', '', 2)]
Entities []
Tokens [('Who', '', 2), ('wanna', '', 2), ('go', '', 2), ('see', '', 2), ('the', '', 2), ('Joker', '', 2), ('movie', '', 2), ('this', '', 2), ('weekend', '', 2), ('?', '', 2), ('#', '', 2), ('JokerMovie', '', 2)]
Entities []
Tokens [('Fan', '', 2), ('art', '', 2), ('inspired', '', 2), ('by', '', 2), ('@jokermovie', '', 2), ('.', '', 2), ('I', '', 2), ('was', '', 2), ('having', '', 2), ('some', '', 2), ('"', '', 2), ('colada', '', 2), ('morada', '', 2), ('"', '', 2), ('and', '', 2), ('decided', '', 2), ('to', '', 2), ('play', '', 2), ('around', '', 2), ('with', '', 2), ('

Entities []
Tokens [('how', '', 2), ('y’', '', 2), ('all', '', 2), ('think', '', 2), ('that', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('going', '', 2), ('to', '', 2), ('be', '', 2), ('?', '', 2), ('I', '', 2), ('want', '', 2), ('to', '', 2), ('see', '', 2), ('it', '', 2), ('myself', '', 2)]
Entities [('character', 'CHARACTERS')]
Tokens [('Wow', '', 2), ('.', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('was', '', 2), ('so', '', 2), ('gruesome', '', 2), ('but', '', 2), ('true', '', 2), ('to', '', 2), ('the', '', 2), ('character', 'CHARACTERS', 3), ('.', '', 2), ('Can', '', 2), ('Christopher', '', 2), ('Nolan', '', 2), ('call', '', 2), ('Bale', '', 2), ('up', '', 2), ('and', '', 2), ('finish', '', 2), ('the', '', 2), ('#', '', 2), ('Joker', '', 2), ('storyline', '', 2), ('now', '', 2), ('.', '', 2), ('I', '', 2), ('’d', '', 2), ('pay', '', 2), ('thru', '', 2), ('the', '', 2), ('roof', '', 2), ('to', '', 2), ('see', '', 2), ('that', '', 2), ('.', '', 2), ('#', '', 2), ('Joker', '', 

Entities [('review', 'REVIEWS')]
Tokens [('This', '', 2), ('week', '', 2), ('all', '', 2), ('about', '', 2), ('the', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('my', '', 2), ('review', 'REVIEWS', 3), ('goes', '', 2), ('up', '', 2), ('Thursday', '', 2), ('night', '', 2), ('pic.twitter.com/HMlpyIDSRe', '', 2)]
Entities [('Joaquin Phoenix', 'TALENT')]
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('A', '', 2), ('thread', '', 2), ('.', '', 2), ('This', '', 2), ('film', '', 2), ('has', '', 2), ('left', '', 2), ('me', '', 2), ('with', '', 2), ('lingering', '', 2), (',', '', 2), ('startling', '', 2), ('images', '', 2), ('24', '', 2), ('hours', '', 2), ('after', '', 2), ('seeing', '', 2), ('it', '', 2), ('.', '', 2), ('Yes', '', 2), (',', '', 2), ('Joaquin', 'TALENT', 3), ('Phoenix', 'TALENT', 1), ('can', '', 2), ('take', '', 2), ('the', '', 2), ('bloody', '', 2), ('joker', '', 2), ('crown', '', 2), ('.', '', 2), ('But', '', 2), ('...', '', 2), ('pic.twitter.com/T5MaDZwUqp', '', 2)]
Entities

Entities []
Tokens [('Was', '', 2), ('really', '', 2), ('excited', '', 2), ('to', '', 2), ('go', '', 2), ('see', '', 2), ('#', '', 2), ('JokerMovie', '', 2), ('on', '', 2), ('my', '', 2), ('birthday', '', 2), ('.', '', 2), ('Now', '', 2), (',', '', 2), ('some', '', 2), ('fucking', '', 2), ('incels', '', 2), ('have', '', 2), ('ruined', '', 2), ('my', '', 2), ('good', '', 2), ('time', '', 2), ('.', '', 2), ('\n\n', '', 2), ('It', '', 2), ('was', '', 2), ('LITERALLY', '', 2), ('my', '', 2), ('only', '', 2), ('birthday', '', 2), ('plan', '', 2), ('.', '', 2)]
Entities []
Tokens [('Upset', '', 2), ('the', '', 2), ('established', '', 2), ('order', '', 2), (',', '', 2), ('and', '', 2), ('everything', '', 2), ('becomes', '', 2), ('chaos', '', 2), ('.', '', 2), ('Todd', '', 2), ('Phillips', '', 2), ('is', '', 2), ('an', '', 2), ('agent', '', 2), ('of', '', 2), ('chaos', '', 2), ('.', '', 2), ('Oh', '', 2), (',', '', 2), ('and', '', 2), ('you', '', 2), ('know', '', 2), ('the', '', 2), ('thing', 

Entities []
Tokens [('#', '', 2), ('JokerMovie', '', 2), ('It', '', 2), ('shows', '', 2), ('how', '', 2), ('everyone', '', 2), ('-', '', 2), ('society', '', 2), (',', '', 2), ('the', '', 2), ('system', '', 2), (',', '', 2), ('bad', '', 2), ('parenting', '', 2), (',', '', 2), ('&', '', 2), ('entertainment', '', 2), ('-', '', 2), ('are', '', 2), ('to', '', 2), ('blame', '', 2), ('for', '', 2), ('the', '', 2), ('mental', '', 2), ('health', '', 2), ('epidemic', '', 2), ('.', '', 2), ('Outside', '', 2), ('of', '', 2), ('maybe', '', 2), ('a', '', 2), ('few', '', 2), ('scenes', '', 2), (',', '', 2), ('I', '', 2), ('felt', '', 2), ('no', '', 2), ('sympathy', '', 2), ('for', '', 2), ('The', '', 2), ('Joker', '', 2), ('.', '', 2), ('Like', '', 2), ('many', '', 2), ('with', '', 2), ('emotional', '', 2), ('struggles', '', 2), (',', '', 2), ('he', '', 2), ('is', '', 2), ("n't", '', 2), ('the', '', 2), ('only', '', 2), ('one', '', 2), ('who', '', 2), ('had', '', 2), ('a', '', 2), ('hand', '', 2), ('

FileNotFoundError: [Errno 2] No such file or directory: 'output/model'